In [45]:
import pandas as pd

file = "confidence_singleterm_dynamic_fewshot_3.5-turbo_gpt-3.5-turbo"
INPUT_PATH = f"../../../Datasets/Evaluations/Sentiment Analysis/Error_Analysis/{file}.csv"
OUTPUT_PATH = f"../../../Datasets/Evaluations/Sentiment Analysis/Error_Analysis/Eval/confidence_analysis_{file}.csv"

In [39]:
df = pd.read_csv(INPUT_PATH)

In [40]:
print(df.head())

   Unnamed: 0.1  Unnamed: 0  text_id  term_id  \
0             0           0        0        0   
1             1           1        0        1   
2             2           2        0        2   
3             3           3        0        3   
4             4           4        1        4   

                                                text  \
0  Amusing details distinguish desserts, from dul...   
1  Amusing details distinguish desserts, from dul...   
2  Amusing details distinguish desserts, from dul...   
3  Amusing details distinguish desserts, from dul...   
4  The server was so busy the night we visited th...   

                          term  polarity  example1_term_id  \
0                     desserts  positive              9500   
1     dulce de leche ice-cream   neutral              9500   
2  chocolate sauce tic-tac-toe   neutral              9500   
3            poached pineapple   neutral              9500   
4                       server  negative              1211

In [41]:

def extract_numbers(text):
    confidence = int(text.split("Confidence: ")[1].split("%")[0])
    sentiment = int(text.split("Sentiment: ")[1].split("%")[0])
    return confidence, sentiment

df[['confidence', 'sentiment']] = df['analysis'].apply(lambda x: pd.Series(extract_numbers(x)))


In [42]:
df['error'] = df.apply(lambda row: 'correct' if row['polarity_pred'] == row['polarity'] else 'error', axis=1)
print(df.head())

   Unnamed: 0.1  Unnamed: 0  text_id  term_id  \
0             0           0        0        0   
1             1           1        0        1   
2             2           2        0        2   
3             3           3        0        3   
4             4           4        1        4   

                                                text  \
0  Amusing details distinguish desserts, from dul...   
1  Amusing details distinguish desserts, from dul...   
2  Amusing details distinguish desserts, from dul...   
3  Amusing details distinguish desserts, from dul...   
4  The server was so busy the night we visited th...   

                          term  polarity  example1_term_id  \
0                     desserts  positive              9500   
1     dulce de leche ice-cream   neutral              9500   
2  chocolate sauce tic-tac-toe   neutral              9500   
3            poached pineapple   neutral              9500   
4                       server  negative              1211

In [43]:
import pandas as pd
df_overall = df
df_positive_pred = df[df['polarity_pred'] == 'positive']
df_negative_pred = df[df['polarity_pred'] == 'negative']
df_neutral_pred = df[df['polarity_pred'] == 'neutral']

df_positive = df[df['polarity'] == 'positive']
df_negative = df[df['polarity'] == 'negative']
df_neutral = df[df['polarity'] == 'neutral']

dfs = [df_overall, df_positive_pred, df_negative_pred, df_neutral_pred, df_positive, df_negative, df_neutral]
i = 1
for df_temp in dfs:
    dfs = [('df_overall', df_overall), ('df_positive_pred', df_positive_pred), ('df_negative_pred', df_negative_pred), ('df_neutral_pred', df_neutral_pred), ('df_positive', df_positive), ('df_negative', df_negative), ('df_neutral', df_neutral)]

# Create an empty dataframe to store the printed values
result_df = pd.DataFrame(columns=['DataFrame'])



In [44]:
# Iterate over the dfs list
for df_name, df_temp in dfs:
    # Create a dictionary to store the printed values
    values = {
        'DataFrame': df_name,
        'Confidence overall mean': df_temp['confidence'].mean(),
        'Confidence overall Std': df_temp['confidence'].std(),
        'Confidence for correct predictions': df_temp[df_temp['error'] == 'correct']['confidence'].mean(),
        'Confidence Std for correct predictions': df_temp[df_temp['error'] == 'correct']['confidence'].std(),
        'Confidence for wrong predictions': df_temp[df_temp['error'] == 'error']['confidence'].mean(),
        'Confidence Std for wrong predictions': df_temp[df_temp['error'] == 'error']['confidence'].std(),
        'Sentiment overall mean': df_temp['sentiment'].mean(),
        'Sentiment overall Std': df_temp['sentiment'].std(),
        'Sentiment for correct predictions': df_temp[df_temp['error'] == 'correct']['sentiment'].mean(),
        'Sentiment Std for correct predictions': df_temp[df_temp['error'] == 'correct']['sentiment'].std(),
        'Sentiment for wrong predictions': df_temp[df_temp['error'] == 'error']['sentiment'].mean(),
        'Sentiment Std for wrong predictions': df_temp[df_temp['error'] == 'error']['sentiment'].std()
    }
    
    # Append the values to the result dataframe
    result_df = pd.concat([result_df, pd.DataFrame(values, index=[i])])
    i += 1

# Print the updated result dataframe
print(result_df)


          DataFrame  Confidence overall mean  Confidence overall Std  \
1        df_overall                94.212860                5.695137   
2  df_positive_pred                96.688312                2.927265   
3  df_negative_pred                95.884956                4.290255   
4   df_neutral_pred                91.114130                6.692547   
5       df_positive                94.892857                5.380081   
6       df_negative                94.102564                6.486570   
7        df_neutral                93.788660                5.381958   

   Confidence for correct predictions  Confidence Std for correct predictions  \
1                           94.650350                                5.117920   
2                           96.767677                                2.972075   
3                           95.800000                                4.429630   
4                           92.008929                                5.868676   
5                 

In [48]:
result_df.to_csv(OUTPUT_PATH, index=False)